In [1]:
import numpy as np
from obspy import read_events
from obspy import read_inventory
from obspy.clients.fdsn import Client
client_wm = Client("IRIS")
from obspy.clients.iris import Client  
from obspy.taup import TauPyModel
from obspy import Stream, Trace
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from obspy import UTCDateTime
import matplotlib.pyplot as plt
from obspy.geodetics import locations2degrees,degrees2kilometers
from scipy import signal
plt.rc('font', size=10) 
%matplotlib widget
import datetime
from obspy.signal.trigger import recursive_sta_lta,plot_trigger,coincidence_trigger
import pandas as pd

In [2]:
inv0 = client_wm.get_stations(
    channel="BH*", starttime=UTCDateTime(2020,9,1,0,0,0),endtime=UTCDateTime(2020,10,1,0,0,0),
    network="AK", sta="SCRK,J25K,K27K",
    level="response")
the_station=inv0[0][2]
evid_dum='https://earthquake.usgs.gov/fdsnws/event/1/query?&starttime=2020-09-01&endtime=2020-9-10&latitude=63.98&longitude=-143.99&maxradius=4&minmagnitude=0'
cat1 = read_events(evid_dum)
evid_dum='https://earthquake.usgs.gov/fdsnws/event/1/query?&starttime=2020-09-01&endtime=2020-9-10&latitude=63.98&longitude=-143.99&minradius=4&maxradius=8&minmagnitude=1'
cat2 = read_events(evid_dum)
evid_dum='https://earthquake.usgs.gov/fdsnws/event/1/query?&starttime=2020-09-01&endtime=2020-9-10&latitude=63.98&longitude=-143.99&minradius=8&maxradius=20&minmagnitude=2'
cat3 = read_events(evid_dum)
evid_dum='https://earthquake.usgs.gov/fdsnws/event/1/query?&starttime=2020-09-01&endtime=2020-9-10&latitude=63.98&longitude=-143.99&minradius=20&maxradius=100&minmagnitude=3'
cat4 = read_events(evid_dum)
evid_dum='https://earthquake.usgs.gov/fdsnws/event/1/query?&starttime=2020-09-01&endtime=2020-9-10&latitude=63.98&longitude=-143.99&minradius=100&maxradius=180&minmagnitude=4'
cat5 = read_events(evid_dum)
cat_all=cat1+cat2+cat3+cat4+cat5
channel = "BHZ"
start_time=UTCDateTime(2020,9,1,0,0,0)
end_time=UTCDateTime(2020,9,10,0,0,0)
#st_2day = client_wm.get_waveforms("AK", "SCRK,K27K,J25K", "*", "BHZ", start_time, end_time)
st_10day = client_wm.get_waveforms("AK", "SCRK", "*", "BHZ", start_time, end_time)
EQ_data=pd.read_csv("detections/SCRK_outputs/X_prediction_results.csv")
eq_arv_t=EQ_data['event_start_time']
eq_arv_s_t=EQ_data['s_arrival_time']
st_10day2=st_10day.copy()
st_10day2.merge(method=0,fill_value=0)
st_10day2.detrend()
st_10day2.filter('bandpass',freqmin=0.8, freqmax=25.0)
trace=st_10day2[0]
model = TauPyModel(model="prem")
df = trace.stats.sampling_rate
npts = trace.stats.npts
t = np.arange(npts, dtype=np.float32) / df
cft = recursive_sta_lta(trace.data, int(5 * df), int(10 * df))
eqt_array=[]
for eqtm in eq_arv_t:
    dumt=UTCDateTime(eqtm)
    #ax.vlines(dumt-trace.stats.starttime, ii,jj, color='y',linestyles='dashdot', lw=3)
    eqt_array.append(dumt-trace.stats.starttime)
    picker_time_a=[]
for event in cat_all:
    disD=locations2degrees(the_station.latitude,the_station.longitude,event.origins[0].latitude,event.origins[0].longitude)
    if event.origins[0].depth<0:
        event.origins[0].depth=0
    arrivals = model.get_travel_times(source_depth_in_km=event.origins[0].depth/1000,
                                  distance_in_degree=disD)    
    picker_time_a.append(event.origins[0].time+arrivals[0].time-trace.stats.starttime)

/Users/yuantian/anaconda3/envs/obspy/lib/python3.7/site-packages/obspy/signal/filter.py:67: UserWarning: Selected high corner frequency (25.0) of bandpass is at or above Nyquist (25.0). Applying a high-pass instead.
  warnings.warn(msg)


In [3]:
max_search_range_b=10
max_search_range_a=30
cft_thr=1.35
# fig = plt.figure(figsize=(10, 5))
# ax = fig.add_subplot(211)
# plt.plot(t,trace.data,'k')
#plt.rc('font', size=20)   
#a,b =ax.get_xlim()
#ii,jj =ax.get_ylim()
sel_cat_list=[]
#ax.set_xlim(0,100000)
lij=-1
for event in cat_all:
    lij=lij+1
    disD=locations2degrees(the_station.latitude,the_station.longitude,event.origins[0].latitude,event.origins[0].longitude)
    picker_time=picker_time_a[lij]
    t0=int(picker_time/trace.stats.delta)
    tarray=range(int(t0-max_search_range_b*trace.stats.sampling_rate),int(t0+max_search_range_a*trace.stats.sampling_rate))
    try:
        if np.max(cft[tarray])>cft_thr:
            #ax.vlines(picker_time, ii,jj, color='r', lw=3)
            sel_cat_list.append(event)
            #ax.text(picker_time,20,"dis=%.2f \n mag=%.2f " % (disD,event.magnitudes[0].mag))
            # if flag==0:
            #     cat_all_sel=event
            #     flag=1
            # else:
            #     cat_all_sel=cat_all_sel+event
    except Exception:
        continue
    
#     ax.vlines(picker_time, ii,jj, color='b', lw=2,linestyles='dashed',label="event origin time")

# for eqtm in eq_arv_t:
#     dumt=UTCDateTime(eqtm)
#     ax.vlines(dumt-trace.stats.starttime, ii,jj, color='y',linestyles='dashdot', lw=3)

# ax2 = fig.add_subplot(212,sharex=ax)
# plt.plot(t,cft,'k')
# ax2.hlines(cft_thr,0,2000000, color='r')
# ax2.set_title('Recursive STA/LTA')
# ax2.set_xlabel("Time after %s [s]" % trace.stats.starttime.isoformat())
# plt.subplots_adjust(hspace=0.3)

In [4]:
for i,event in enumerate(sel_cat_list):
    if i==0:
        dum=cat_all[0:1]
        dum.clear()
        all_sel_event=dum+event
    else:
        all_sel_event=all_sel_event+event
lij=-1
eqt_max_time=25
for i,picker_time in enumerate(picker_time_a):
    if np.abs(np.array(eqt_array)-picker_time).min()<eqt_max_time:
        lij=lij+1
        if lij==0:
            dum=cat_all[0:1]
            dum.clear()
            eqt_sel_event=dum+cat_all[i]
        else:
            eqt_sel_event=eqt_sel_event+cat_all[i]
eqtpall_sel_event=all_sel_event+eqt_sel_event
event_in_list=[]
for event in eqtpall_sel_event:
    disD=locations2degrees(the_station.latitude,the_station.longitude,event.origins[0].latitude,event.origins[0].longitude)
    event_list=[event.origins[0].longitude,event.origins[0].latitude,event.origins[0].depth,event.magnitudes[0].mag,disD,event.event_type,event.origins[0].time]
    event_in_list.append(event_list)
eqtpall_sel_event_pd=pd.DataFrame(event_in_list,columns=['longitude','latitude','depth','magnitude','disD','event_type','time'])
bh_catalog=eqtpall_sel_event_pd.drop_duplicates(subset=['longitude','latitude','magnitude'])

In [5]:
bh_catalog

,longitude,latitude,depth,magnitude,disD,event_type,time
0,-146.8571,62.8881,44100.0,2.0,1.681262,earthquake,2020-09-09T19:12:06.535000Z
1,-146.8485,62.8784,45600.0,2.5,1.684798,earthquake,2020-09-09T19:02:13.756000Z
2,-147.7625,63.5276,10800.0,0.8,1.727156,earthquake,2020-09-09T15:29:05.744000Z
3,-148.3746,63.7534,84600.0,1.2,1.943567,earthquake,2020-09-09T11:12:10.065000Z
4,-149.0041,62.7214,62500.0,2.4,2.574223,earthquake,2020-09-09T10:00:10.896000Z
...,...,...,...,...,...,...,...
199,-71.3062,-27.9686,21000.0,6.8,107.824508,earthquake,2020-09-01T04:09:28.470000Z
204,-139.7911,64.8705,7600.0,1.2,2.021046,earthquake,2020-09-09T02:02:00.010000Z
206,-147.7667,64.7974,11700.0,0.9,1.827003,earthquake,2020-09-07T21:47:51.243000Z
217,-141.8599,65.8902,10800.0,2.0,2.115990,earthquake,2020-09-04T23:44:01.585000Z


In [6]:
def threshold_trigger(th,record_time,trace):
    record_number=record_time*trace.stats.sampling_rate
    abs_st_data=np.abs(trace.data)
    ind_th=[]
    recording=False
    for i,abs_st in enumerate(abs_st_data):
        if abs_st>th and recording is False:
            ind_th.append(i)
            recording=True
        if ind_th!=[] and (i-ind_th[-1])>record_number:
            recording=False
    return len(ind_th),ind_th

def cal_bhcat_t(bh_catalog):
    cat_arl_t=[]
    cat_arl_i=[]
    for i in bh_catalog.index:
        arrivals = model.get_travel_times(source_depth_in_km=bh_catalog.loc[i,'depth']/1000,
                                    distance_in_degree=bh_catalog.loc[i,'disD'])    
        picker_time=bh_catalog.loc[i,'time']+arrivals[0].time-trace.stats.starttime
        cat_arl_t.append(picker_time)
        cat_arl_i.append(i)
    return cat_arl_t,cat_arl_i


def event_associate(pre_time,after_time,cat_arl_t,t,ind_th,cat_arl_i):
    n_recorded=0
    index_triggered=[]
    index_cat=[]
    for i in ind_th:
        for j,cat_t in enumerate(cat_arl_t):
            if cat_t>t[i]-pre_time and cat_t<t[i]+after_time:
                n_recorded=n_recorded+1
                index_triggered.append(cat_arl_i[j])
                index_cat.append(j)
                break
    return n_recorded,index_triggered,index_cat
def threshold_trigger_w_avg(th_fold,record_time,trace,window_len):
    record_number=record_time*trace.stats.sampling_rate
    abs_st_data=np.abs(trace.data)
    ind_th=[]
    recording=False
    rtt=0
    cm_sum=0
    ltavg=np.sum(abs_st_data[0:window_len])/window_len
    for i,abs_st in enumerate(abs_st_data):
        rtt=rtt+1
        if rtt>window_len:
            rtt=0
            ltavg=cm_sum/window_len
            #print(ltavg)
            cm_sum=abs_st
        cm_sum=cm_sum+abs_st
        if abs_st>th_fold*ltavg and recording is False:
            ind_th.append(i)
            recording=True
        if ind_th!=[] and (i-ind_th[-1])>record_number:
            recording=False
    return len(ind_th),ind_th

In [7]:
cat_arl_t,cat_arl_i=cal_bhcat_t(bh_catalog)

Performance compare

In [8]:
#st_10day2.filter('bandpass',freqmin=0.8, freqmax=25.0)
fmin=2
fmax=8
st_10day2=st_10day.copy()
st_10day2.merge(method=0,fill_value=0)
st_10day2.detrend()
st_10day2.filter('bandpass',freqmin=fmin, freqmax=fmax)
trace=st_10day2[0]
pre_time=80
after_time=10
th_fold=200
th=200
record_time=60*30
fminay=np.arange(0.02,10,0.4)
window_len=50*60*2
#th_a=range(30,210,30)
th_a=[30,40,50,70,90,120,150,180,200]
n_thres=[]
n_tr_thres=[]
pect_thres=[]
n_event=[]
for th in th_a:
    n_dum,ind_th=threshold_trigger(th,record_time,trace)
    n_recorded,index_triggered,index_cat=event_associate(pre_time,after_time,cat_arl_t,t,ind_th,cat_arl_i)
    n_thres.append(n_recorded)
    n_tr_thres.append(n_dum)
    n_event.append(len(cat_arl_t))
    pect_thres.append(n_recorded/n_dum)
plt.figure()
plt.plot(n_tr_thres,pect_thres)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
pre_time=80
after_time=10
th_fold=200
th=200
record_time=60*30
rest_time=30*60
window_len=50*60*2
th_a=range(20,200,10)
n_sw=[]
n_tr_sw=[]
pect_sw=[]
n_event_sw=[]
for th in th_a:
    #n_dum,ind_th=threshold_trigger(th,record_time,trace)
    n_dum,ind_th=threshold_trigger_w_avg(th,rest_time,trace,window_len)
    n_recorded,index_triggered,index_cat=event_associate(pre_time,after_time,cat_arl_t,t,ind_th,cat_arl_i)
    n_sw.append(n_recorded)
    n_tr_sw.append(n_dum)
    n_event_sw.append(len(cat_arl_t))
    pect_sw.append(n_recorded/n_dum)
plt.figure()
plt.plot(n_tr_sw,pect_sw)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
def threshold_trigger_cft(th,record_time,trace,cft):
    record_number=record_time*trace.stats.sampling_rate
    abs_st_data=cft
    ind_th=[]
    recording=False
    for i,abs_st in enumerate(abs_st_data):
        if abs_st>th and recording is False:
            ind_th.append(i)
            recording=True
        if ind_th!=[] and (i-ind_th[-1])>record_number:
            recording=False
    return len(ind_th),ind_th

In [20]:
cft
#st_10day2.filter('bandpass',freqmin=0.8, freqmax=25.0)
fmin=0.8
fmax=25
st_10day2=st_10day.copy()
st_10day2.merge(method=0,fill_value=0)
st_10day2.detrend()
st_10day2.filter('bandpass',freqmin=fmin, freqmax=fmax)
trace=st_10day2[0]
pre_time=80
after_time=10
th_fold=200
th=200
record_time=60*30
fminay=np.arange(0.02,10,0.4)
window_len=50*60*2
#th_a=range(30,210,30)
#th_a=[30,40,50,70,90,120,150,180,200]
th_a=np.arange(1.57,1.99,0.03)
cft = recursive_sta_lta(trace.data, int(5 * df), int(10 * df))
n_thres_cft=[]
n_tr_thres_cft=[]
pect_thres_cft=[]
n_event_cft=[]
for th in th_a:
    n_dum,ind_th=threshold_trigger_cft(th,record_time,trace,cft)
    n_recorded,index_triggered,index_cat=event_associate(pre_time,after_time,cat_arl_t,t,ind_th,cat_arl_i)
    n_thres_cft.append(n_recorded)
    n_tr_thres_cft.append(n_dum)
    n_event_cft.append(len(cat_arl_t))
    pect_thres_cft.append(n_recorded/n_dum)
plt.figure()
plt.plot(n_tr_thres_cft,pect_thres_cft)

/Users/yuantian/anaconda3/envs/obspy/lib/python3.7/site-packages/obspy/signal/filter.py:67: UserWarning: Selected high corner frequency (25) of bandpass is at or above Nyquist (25.0). Applying a high-pass instead.
  warnings.warn(msg)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
plt.figure(figsize=(8,10))
plt.subplot(3,1,1)
plt.plot(n_tr_sw,pect_sw,'r-o')
plt.plot(n_tr_thres,pect_thres,'k-o')
plt.plot(n_tr_thres_cft,pect_thres_cft,'b-o')
plt.ylabel('true positive rate')
#plt.xlabel('number of triggers')
plt.legend(['Segmented window','Threshold','STA/LTA'])
plt.subplot(3,1,2)
plt.plot(n_tr_sw,1-np.array(pect_sw),'r-o')
plt.plot(n_tr_thres,1-np.array(pect_thres),'k-o')
plt.plot(n_tr_thres_cft,1-np.array(pect_thres_cft),'b-o')
plt.ylabel('flase positive rate')
#plt.xlabel('number of triggers')
plt.legend(['Segmented window','Threshold','STA/LTA'])

plt.subplot(3,1,3)
plt.plot(n_tr_sw,(n_event_cft[0]-np.array(n_sw))/n_event_cft[0],'r-o')
plt.plot(n_tr_thres,(n_event_cft[0]-np.array(n_thres))/n_event_cft[0],'k-o')
plt.plot(n_tr_thres_cft,(n_event_cft[0]-np.array(n_thres_cft))/n_event_cft[0],'b-o')
plt.ylabel('true negative rate')
plt.xlabel('number of triggers')
plt.legend(['Segmented window','Threshold','STA/LTA'])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …